In [3]:
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair
from charm.toolbox.ABEnc import ABEnc
from abenc_tg22 import DIPE_ABE, Inner_Product_TG22
from abenc_ph_mj18 import PH_ABE, mat_math, Inner_Product
from nizk import NIZK
from attack import RogueKeyAtt
from collections import defaultdict 
import time
group_tg = PairingGroup('SS512')
group_mj = PairingGroup('MNT159')

/home/guy1m0/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [13]:
def benchmark(assump_size, n, datasets):

    # Pre-processing
    tg = {}
    mj = {}

    mj['total'] = 0
    tg['total'] = 0
    dipe = DIPE_ABE(n, group_tg)
    ph_abe = PH_ABE(n, assump_size, group_mj, mat_math())

    nizk = NIZK(group_mj)
    att = RogueKeyAtt(n, assump_size)

    print ('Setting with n:', n, "k:",assump_size)

    # sys setup
    start_time = time.time()
    pp_mj, msk_mj = ph_abe.setup()
    elapsed_time = time.time() - start_time
    mj['sys'] = elapsed_time
    mj['total'] += elapsed_time

    start_time = time.time()
    pp_tg = dipe.setup()
    elapsed_time = time.time() - start_time
    tg['sys'] = elapsed_time
    tg['total'] += elapsed_time


    # Auth setup
    start_time = time.time()
    pks_mj, sks_mj = ph_abe.auth_setup(pp_mj)
    elapsed_time = time.time() - start_time
    mj['auth'] = elapsed_time
    mj['total'] += elapsed_time

    start_time = time.time()
    pks_tg, sks_tg = dipe.auth_setup(pp_tg)
    elapsed_time = time.time() - start_time
    tg['auth'] = elapsed_time
    tg['total'] += elapsed_time

    # Key Gen
    GID_tg = group_tg.random(ZR)
    GID_mj = group_mj.random(ZR)

    vec_x, vec_y = Inner_Product_TG22(group_tg).gen_x_y(n)
    start_time = time.time()
    D0, D1, K_tg = dipe.keygen(pp_tg, sks_tg, GID_tg, vec_x)
    elapsed_time = time.time() - start_time
    tg['keygen'] = elapsed_time
    tg['total'] += elapsed_time

    vec_x, vec_v = Inner_Product(group_mj).gen_x_v(n, assump_size)

    start_time = time.time()
    K_mj, _ = ph_abe.keygen(pp_mj, pks_mj, sks_mj, GID_mj, vec_v)
    elapsed_time = time.time() - start_time
    mj['keygen'] = elapsed_time
    mj['total'] += elapsed_time

    # Worst Case verify/prove
    start_time = time.time()
    for i in range(n):
        s_pairs, pis = nizk.prove_pk(pp_mj, pks_mj[str(i+1)], sks_mj[str(i+1)])
    elapsed_time = time.time() - start_time
    mj['prove'] = elapsed_time

    start_time = time.time()
    for i in range(n):
        check = nizk.verify_pk(pp_mj, s_pairs, pis)
        if not check:
            print ("Check not pass for AA:", str(i+1))
            break
    elapsed_time = time.time() - start_time
    mj['verify'] = elapsed_time

    # AD's setup
    start_time = time.time()
    ad = vec_v.index(0) + 1

    ad_vec_v = [0] * (n-1) + [1]
    pks_mj = att.pks_update(ad, pks_mj)
    elapsed_time = time.time() - start_time
    mj['ad_setup'] = elapsed_time
    #print ("Adv's setup: ", elapsed_time)

    # AD KeyGen
    GID_ad = group_mj.random(ZR)
    K_,ad_key_gen = ph_abe.keygen(pp_mj, pks_mj, sks_mj, GID_ad, ad_vec_v, ad=ad)
    mj['ad_keygen'] = ad_key_gen

    # Encryption
    M_tg = group_tg.random(GT)
    start_time = time.time()
    C_tg = dipe.encrypt(pp_tg, pks_tg, vec_y, M_tg)
    elapsed_time = time.time() - start_time
    tg['encrypt'] = elapsed_time
    tg['total'] += elapsed_time

    M_mj = group_mj.random(GT)
    start_time = time.time()    
    #print ('M:', M)
    C_mj, vec_s = ph_abe.encrypt(pp_mj, pks_mj, vec_x, M_mj)
    elapsed_time = time.time() - start_time
    mj['encrypt'] = elapsed_time
    mj['total'] += elapsed_time

    # Decryption
    start_time = time.time()
    M_ = dipe.decrypt(D0, D1, K_tg, C_tg, vec_y)
    if M_ != M_tg:
        print ('Error in decrypt M (usr): ', M_)
    elapsed_time = time.time() - start_time
    #print ("DU's cost: ", elapsed_time)
    tg['decrypt'] = elapsed_time
    tg['total'] += elapsed_time


    start_time = time.time()
    M_ = ph_abe.decrypt(K_mj, C_mj, vec_v, pp_mj)
    if M_ != M_mj:
        print ('Error in decrypt M (usr): ', M_)
    elapsed_time = time.time() - start_time
    #print ("DU's cost: ", elapsed_time)
    mj['decrypt'] = elapsed_time
    mj['total'] += elapsed_time

    # AD decrypt
    start_time = time.time()
    tmp = ph_abe.decrypt(K_, C_mj, ad_vec_v, pp_mj)
    omega = att.gen_omega(K_,C_mj)
    M_ =  tmp * mat_math().prod(omega)
    if M_ != M_mj:
        print ('Error in decrypt M (adv): ', M_)
    elapsed_time = time.time() - start_time
    mj['ad_cancel_out'] = elapsed_time
    #print ("Ad's cost: ", data['ad_setup'] + data['ad_keygen'] + data['ad_cancel_out'])

    print ("MJ's sys cost: ", mj['total'])
    print ("TG's sys cost: ", tg['total'])
    print ("Enhd sys cost: ", mj['total'] + mj['prove'] + mj['verify'])

    datasets['k_' + str(assump_size)+'_n_'+str(n)]= {'mj':mj, 'tg':tg}

    return datasets
    # datasets[str(k)]['n']=data
    # datasets[str(k)]['seq'].append(data)




In [14]:
datasets = {}
datasets = benchmark(1, 10, datasets)

Setting with n: 10 k: 1
MJ's sys cost:  0.28943634033203125
TG's sys cost:  0.17408394813537598
Enhd sys cost:  0.4365425109863281
